In [1]:
import pandas as pd
import os
import requests
import json
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../Resources/Serie_A/Italy_city.csv")
df.head()
# Date string format
def Format_Date(df):
    # df is series with the key “Date”
    # expressed as month/day/year
    df2 = df.copy()
    one_digit = [str(i) for i in range(10)]
    for i, val in df.items():
        dates = val.split("/")
        if dates[0] in one_digit:
            dates[0] = "0"+dates[0]
        if dates[1] in one_digit:
            dates[1] = "0"+dates[1]
        if dates[2] == "19" or dates[2] == "20":
            dates[2] = "20"+dates[2]
        df2.loc[i] = dates[2]+"-"+dates[1]+"-"+dates[0]
    return df2
df["Date"] = Format_Date(df["Date"])
weather_df = df
weather_df.head()

,Div,Date,Time,HomeTeam,AwayTeam,HomeScore,AwayScore,Result,HomeTeam ID,AwayTeam ID,City,Unnamed: 11
0,I1,2019-08-24,17:00,Parma,Juventus,0,1,A,SER12,SER7,Parma,NaN
1,I1,2019-08-24,19:45,Fiorentina,Napoli,3,4,A,SER4,SER11,Florence,NaN
2,I1,2019-08-25,17:00,Udinese,Milan,1,0,H,SER18,SER10,Udine,NaN
3,I1,2019-08-25,19:45,Cagliari,Brescia,0,1,A,SER3,SER2,Cagliari,NaN
4,I1,2019-08-25,19:45,Roma,Genoa,3,3,D,SER13,SER5,Rome,NaN


In [3]:
cities = []
cities = weather_df["City"]
lat = []
lng = []

In [4]:
query = "https://maps.googleapis.com/maps/api/geocode/json"
from config import gkey
# Exception Handling
for row in cities:
        try:
                params = {"address": str(row)+","+"Italy", "key": gkey}
                response = requests.get(query,params=params).json()
                lat.append(response["results"][0]["geometry"]["location"]["lat"])
                lng.append(response["results"][0]["geometry"]["location"]["lng"])
        except: 
                print("Oops")


In [5]:
weather_df["Latitude"] = lat
weather_df["Longitude"] = lng 
weather_df

,Div,Date,Time,HomeTeam,AwayTeam,HomeScore,AwayScore,Result,HomeTeam ID,AwayTeam ID,City,Unnamed: 11,Latitude,Longitude
0,I1,2019-08-24,17:00,Parma,Juventus,0,1,A,SER12,SER7,Parma,NaN,44.801485,10.327904
1,I1,2019-08-24,19:45,Fiorentina,Napoli,3,4,A,SER4,SER11,Florence,NaN,43.769560,11.255814
2,I1,2019-08-25,17:00,Udinese,Milan,1,0,H,SER18,SER10,Udine,NaN,46.071067,13.234579
3,I1,2019-08-25,19:45,Cagliari,Brescia,0,1,A,SER3,SER2,Cagliari,NaN,39.223841,9.121661
4,I1,2019-08-25,19:45,Roma,Genoa,3,3,D,SER13,SER5,Rome,NaN,41.902783,12.496365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,I1,2020-07-12,18:30,Fiorentina,Verona,1,1,D,SER4,SER19,Florence,NaN,43.769560,11.255814
316,I1,2020-07-12,18:30,Parma,Bologna,2,2,D,SER12,SER1,Parma,NaN,44.801485,10.327904
317,I1,2020-07-12,18:30,Udinese,Sampdoria,1,3,A,SER18,SER14,Udine,NaN,46.071067,13.234579
318,I1,2020-07-12,20:45,Napoli,Milan,2,2,D,SER11,SER10,Naples,NaN,40.851798,14.268120


In [6]:
url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx"
from config import weather_key2
key = weather_key2
averagetemp = []
formatver = "json"
for index,row in weather_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    date = row["Date"]
    q = str(lat) + "," + str(lng)
    params = {"q": q,"format": formatver,"key" : key,"date": date}
    response = requests.get(url,params=params).json()
    averagetemp.append(response["data"]["weather"][0]["avgtempF"])


In [ ]:
print(averagetemp)

In [7]:
weather_df["Avg Temp (in F)"] = averagetemp
weather_df

,Div,Date,Time,HomeTeam,AwayTeam,HomeScore,AwayScore,Result,HomeTeam ID,AwayTeam ID,City,Unnamed: 11,Latitude,Longitude,Avg Temp (in F)
0,I1,2019-08-24,17:00,Parma,Juventus,0,1,A,SER12,SER7,Parma,NaN,44.801485,10.327904,79
1,I1,2019-08-24,19:45,Fiorentina,Napoli,3,4,A,SER4,SER11,Florence,NaN,43.769560,11.255814,85
2,I1,2019-08-25,17:00,Udinese,Milan,1,0,H,SER18,SER10,Udine,NaN,46.071067,13.234579,82
3,I1,2019-08-25,19:45,Cagliari,Brescia,0,1,A,SER3,SER2,Cagliari,NaN,39.223841,9.121661,81
4,I1,2019-08-25,19:45,Roma,Genoa,3,3,D,SER13,SER5,Rome,NaN,41.902783,12.496365,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,I1,2020-07-12,18:30,Fiorentina,Verona,1,1,D,SER4,SER19,Florence,NaN,43.769560,11.255814,80
316,I1,2020-07-12,18:30,Parma,Bologna,2,2,D,SER12,SER1,Parma,NaN,44.801485,10.327904,75
317,I1,2020-07-12,18:30,Udinese,Sampdoria,1,3,A,SER18,SER14,Udine,NaN,46.071067,13.234579,72
318,I1,2020-07-12,20:45,Napoli,Milan,2,2,D,SER11,SER10,Naples,NaN,40.851798,14.268120,86


In [8]:
weather_df.to_csv("../final-resources/Serie_A_weather.csv")